In [1]:
%%capture
!pip install bertopic

In [22]:
import zipfile
import os
import nltk
import pymorphy2 as pm
morph = pm.MorphAnalyzer()
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer 

In [5]:
with zipfile.ZipFile('/content/manuscript.zip', 'r') as zip_ref:
    zip_ref.extractall('manuscript')

In [28]:
corpus_files = os.listdir('manuscript/рукопись')

In [29]:
with open('swl_optimum.txt', encoding = 'utf-8') as f:
  stw_list = f.read().split()

In [33]:
docs = []
dates = []

# Предобрабатываем данные (предобработка одинаковая для всех моделей)
for folder in corpus_files:
  for txt in os.listdir('manuscript/рукопись/' + folder):
    with open('manuscript/рукопись/' + folder + '/' + txt, 'r', encoding='windows-1251') as f:
      doc = []
      text = f.readlines()
      for line in text:
        if '\n' in line:
          line = line.replace('\n', '')
          doc.append(line)
      dates.append(doc.pop(0))
      docs.append(' '.join(doc))

In [35]:
docs[0]

'     Проснувшись чуть свет, я вдруг решил посмотреть  на  страшную  виселицу Лос-Эрманос в надежде опять найти там  какую-нибудь  жертву.  Прогулка  не оказалась напрасной, так как я действительно нашел там человека,  лежащего между двумя  висельниками.  Несчастный  казался  совсем  бесчувственным  и похолодевшим, - однако, дотронувшись до его руки, я убедился, что жизнь  в нем еще теплится. Я принес воды и побрызгал ему в лицо, но,  видя  что  он нисколько не приходит в себя,  взвалил  его  себе  на  плечи  и  вынес  из виселичной ограды. Он медленно очнулся, вперил  в  меня  безумный  взгляд, потом вдруг вырвался и побежал в поле. Некоторое время  я  следил  за  ним глазами, но, видя, что он кинулся в кусты и легко может заблудиться в этой пустынной местности, счел своей обязанностью пуститься за ним  вдогонку  и остановить его. Незнакомец обернулся и, видя, что его преследуют, прибавил ходу; наконец он зашатался, упал и ранил себе голову. Я обтер ему  платком рану,  потом  обвязал 

In [36]:
dates

['ДЕНЬ ВОСЕМНАДЦАТЫЙ',
 'ДЕНЬ СЕМНАДЦАТЫЙ',
 'ДЕНЬ ДЕВЯТНАДЦАТЫЙ',
 'ДЕНЬ ДВАДЦАТЫЙ',
 'ДЕНЬ ШЕСТНАДЦАТЫЙ',
 'ДЕНЬ ДВАДЦАТЬ ПЕРВЫЙ',
 'ДЕНЬ ВОСЕМНАДЦАТЫЙ',
 'ДЕНЬ СОРОК ДЕВЯТЫЙ',
 'ДЕНЬ СОРОК СЕДЬМОЙ',
 'ДЕНЬ СОРОК ТРЕТИЙ',
 'ДЕНЬ СОРОК ЧЕТВЕРТЫЙ',
 'ДЕНЬ СОРОК ПЯТЫЙ',
 'ДЕНЬ СОРОК ШЕСТОЙ',
 'ДЕНЬ СОРОК ВОСЬМОЙ',
 'ДЕНЬ ПЯТЬДЕСЯТ ШЕСТОЙ',
 'ДЕНЬ ПЯТЬДЕСЯТ ЧЕТВЕРТЫЙ',
 'ДЕНЬ ПЯТИДЕСЯТЫЙ',
 'ДЕНЬ ПЯТЬДЕСЯТ ПЯТЫЙ',
 'ДЕНЬ ПЯТЬДЕСЯТ ВТОРОЙ',
 'ДЕНЬ ПЯТЬДЕСЯТ ТРЕТИЙ',
 'ДЕНЬ ПЯТЬДЕСЯТ ПЕРВЫЙ',
 'ДЕНЬ ТРЕТИЙ',
 'ДЕНЬ ПЯТЫЙ',
 'ДЕНЬ ЧЕТВЕРТЫЙ',
 'ДЕНЬ ПЕРВЫЙ',
 'ДЕНЬ ШЕСТОЙ',
 'ДЕНЬ ВТОРОЙ',
 'ДЕНЬ СЕДЬМОЙ',
 'ДЕНЬ ОДИННАДЦАТЫЙ',
 'ДЕНЬ ДЕСЯТЫЙ',
 'ДЕНЬ ВОСЬМОЙ',
 'ДЕНЬ ДЕВЯТЫЙ',
 'ДЕНЬ ТРИНАДЦАТЫЙ',
 'ДЕНЬ ЧЕТЫРНАДЦАТЫЙ',
 'ДЕНЬ ДВЕНАДЦАТЫЙ',
 'ДЕНЬ ПЯТЬДЕСЯТ ДЕВЯТЫЙ',
 'ДЕНЬ ШЕСТЬДЕСЯТ ВТОРОЙ',
 'ДЕНЬ ШЕСТИДЕСЯТЫЙ',
 'ДЕНЬ ШЕСТЬДЕСЯТ ПЕРВЫЙ',
 'ДЕНЬ ШЕСТЬДЕСЯТ ТРЕТИЙ',
 'ДЕНЬ ПЯТЬДЕСЯТ СЕДЬМОЙ',
 'ДЕНЬ ПЯТЬДЕСЯТ ВОСЬМОЙ',
 'ЗАКЛЮЧЕНИЕ',
 'ДЕНЬ ШЕСТЬДЕСЯТ ШЕСТОЙ',
 'ДЕНЬ ШЕСТЬДЕСЯТ ПЯТЫЙ',
 

In [37]:
corpus_texts = []

for text in docs:
  text = nltk.word_tokenize(text.lower())
  text = [token for token in text if len(token) > 2]
  text = [morph.parse(token)[0].normal_form for token in text if token not in stw_list]
  corpus_texts.append(' '.join(text))

# **Dynamic Topic Modeling**


## Basic Topic Model
To perform Dynamic Topic Modeling with BERTopic we will first need to create a basic topic model using all tweets. The temporal aspect will be ignored as we are, for now, only interested in the topics that reside in those tweets. 

In [38]:
vectorizer_model = CountVectorizer(stop_words=stw_list, ngram_range=(1,3)) 
topic_model = BERTopic(language="russian", vectorizer_model=vectorizer_model, calculate_probabilities=True, min_topic_size=3, verbose=True)
topics, _ = topic_model.fit_transform(corpus_texts)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2022-12-21 22:33:53,131 - BERTopic - Transformed documents to Embeddings
2022-12-21 22:33:55,646 - BERTopic - Reduced dimensionality
2022-12-21 22:33:55,664 - BERTopic - Clustered reduced embeddings


We can then extract most frequent topics:

In [39]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name
0,-1,16,-1_быть_отец_сказать_эльвира
1,0,30,0_быть_сказать_отец_день
2,1,12,1_быть_герцог_год_дом
3,2,6,2_быть_религия_понятие_человек
4,3,5,3_быть_шейх_быть отец_отец
5,4,5,4_отец_быть_сказать_сеньор
6,5,3,5_сеньор_сказать_быть_бускероса


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [41]:
topic_model.get_topic(4)

[('отец', 0.017444894990296392),
 ('быть', 0.013671153812336714),
 ('сказать', 0.010740678150761957),
 ('сеньор', 0.010451196188515349),
 ('мать', 0.010250897640420123),
 ('стать', 0.00984877517337656),
 ('время', 0.008795943122991676),
 ('окно', 0.008185381991375026),
 ('мочь', 0.00788687379502005),
 ('дом', 0.007729118583575988)]

We can visualize the basic topics that were created with the Intertopic Distance Map. This allows us to judge visually whether the basic topics are sufficient before proceeding to creating the topics over time. 

In [42]:
fig = topic_model.visualize_topics(); fig

In [44]:
topic_model.visualize_barchart()

## Topics over Time
Before we start with the Dynamic Topic Modeling step, it is important that you are satisfied with the topics that were created previously. We are going to be using those specific topics as a base for Dynamic Topic Modeling. 

Thus, this step will essentially show you how the topics that were defined previously have evolved over time. 

There are a few important parameters that you should take note of, namely:

* `docs`
  * These are the tweets that we are using
* `timestamps`
  * The timestamp of each tweet/document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20. 


In [47]:
time_markers = []
for date in dates:
  time_marker = []
  for word in nltk.word_tokenize(date):
    word = word.lower()
    time_marker.append(word)
  time_markers.append(time_marker[1:])

In [49]:
tens = ['двадцат', 'тридцат', 'сорок', 'пятьдесят', 'пятидесят', 'шестьдесят', 'шестидесят']
numbers = ['первый', 'второй', 'третий', 'четвертый', 'пятый', 'шестой', 'седьмой', 
           'восьмой', 'девятый', 'десятый', 'одиннадцатый', 'двенадцатый', 'тринадцатый', 
           'четырнадцатый', 'пятнадцатый', 'шестнадцатый', 'семнадцатый', 'восемнадцатый', 'девятнадцатый']

In [50]:
for time in time_markers:
  for i, num in enumerate(time):
    for j, ten in enumerate(tens):
      if ten in num:
        time[i] = j+2
    for k, fig in enumerate(numbers):
      if fig in num:
        time[i] = k+1

In [52]:
dates_num = []
for time in time_markers:
  if len(time) > 1:
    time = str(time[0]) + str(time[1])
  else:
    for t in time:
      time = str(t)
  dates_num.append(time)

In [53]:
for i, item in enumerate(dates_num):
  if type(item) is str:
    dates_num[i] = int(item)
  elif type(item) is list:
    dates_num[i] = 77

In [54]:
topics_over_time = topic_model.topics_over_time(docs=corpus_texts, 
                                                timestamps=dates_num, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

18it [00:07,  2.43it/s]


## Visualize Topics over Time
After having created our `topics_over_time`, we will have to visualize those topics as accessing them becomes a bit more difficult with the added temporal dimension. 

To do so, we are going to visualize the distribution of topics over time based on their frequency. Doing so allows us to see how the topics have evolved over time. Make sure to hover over any point to see how the topic representation at time *t* differs from the global topic representation. 


In [55]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)